In [ ]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex
import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta
import scipy as sp 
from scipy.optimize import minimize
from threading import Thread

In [ ]:
# Configuration

# Date Ranges
returns_start = "2024-11-14" 
returns_end = "2025-11-14" 

history_start = "2024-10-01"
history_end = "2025-10-01"

# Volume filtering
min_avg_vol = 5000
min_trading_days_per_month = 18

# Portfolio size constraints
min_port_size = 15
max_port_size = 25

# Market cap constraints (in USD)
small_cap_threshold = 2_000_000_000
large_cap_threshold = 10_000_000_000

# Weight constraints (per stock)
min_weight_pct = None
max_weight_pct = 0.15

# Sector constraints
max_sector_weight = 0.40

# Currency constraints
allowed_currencies = ["CAD", "USD"]

# Budget
total_budget = 1_000_000

# Fee structure
flat_fee_cad = 2.5
variable_fee_per_share = 0.001

In [ ]:
def last_trading_day(date=None):
    """
    Find the last trading day (weekday) before a given date
    """
    if date is None:
        date = datetime.today()
    # Step back one day at a time until it's a weekday, aka Monday or Friday
    while date.weekday() >= 5: # Saturday = 5, Sunday = 6
        date -= timedelta(days=1)
    return date

purchase_date = last_trading_day()

In [ ]:
def valid_tickers(csv_file_path):
   """
   valid_tickers is the prelimary checker, it reads a csv and then removes any tickers that trade less than the minimum volume

   :param csv_file_path: A string that represents the csv file name that will be read
   :return: A list of names of the valid tickers
   """
   
   print(f"Now loading tickers from {csv_file_path}.")

   # Scrape for unique tickers
   try:
      tickers_df = pd.read_csv(csv_file_path, header=None)
      tickers_df.columns = ["Ticker"]
      tickers_df["Ticker"] = tickers_df["Ticker"].str.strip()
      tickers_df = tickers_df.drop_duplicates()
      ticker_list = tickers_df["Ticker"].tolist()

      print(f"{len(ticker_list)} unique tickers have been found in the csv. Now downloading tickers.")
   except FileNotFoundError:
      raise FileNotFoundError(f"Could not find file path to {csv_file_path}.")

   # Bulk download the ticker information
   hist = yf.download(
       tickers=ticker_list,
       start=history_start,
       end=history_end,
       group_by="ticker",
       auto_adjust=False,
       threads=True,
       progress=False
    )
   
   if hist.empty:
      raise ValueError("No data downloaded, check ticker symbols and date range.")

   # Count the rows per month per ticker
   trading_days_per_month = hist.groupby(hist.index.to_period("M")).transform("count")

   # Keep only rows where all tickers in that month exceed configured minimum
   valid_days_mask = trading_days_per_month >= min_trading_days_per_month
   hist_filtered = hist[valid_days_mask]

   # Remove the months that do not meet threshold and any tickers that don't have a large enough trading volume
   vol_filtered = hist_filtered.xs("Volume", axis=1, level=1)
   avg_vol_filtered = vol_filtered.mean()
   valid_tickers = avg_vol_filtered[avg_vol_filtered >= min_avg_vol].index.tolist()

   return valid_tickers


In [ ]:
#Begin threading set up
def get_format_info(ticker, ticker_data):
   """
   get_format_info is the threading worker function, it is what will be done in parallel. It finds wanted ticker info
   and adds it to a list

   :param ticker: The ticker whose info is wanted
   :param ticker_data: A list that keeps track of the ticker info thats been extracted
   """
   ticker_info = yf.Ticker(ticker).info

   # Get the currency, recall we only want USD and CAD traded stocks
   currency = ticker_info.get("currency")
   if (currency not in ["USD", "CAD"]):
      currency = "NaN"
   
   if ticker_info.get("quoteType") == "EQUITY": # ADDED THIS TO FIX CODE
      ticker_data.append((ticker, ticker_info.get("sector"), currency, ticker_info.get("marketCap")))

#Threading occurs
def format_tickers(csv_file_path):
   """
   format_tickers is the main function that performs threading onto the worker function and gets the dataframe of tickers and wanted info

   :param csv_file_path: A string that represents the csv file name that will be read
   :return: A Dataframe that contains all the info we want for every valid ticker
   """
   ticker_list = valid_tickers(csv_file_path)
   
   # Threading set up, ticker_data is the shared list where threads store their results, threads is the list of the thread objects
   ticker_data = []
   threads = []

   # Loop and create threads of each ticker in the ticker_list
   for t in ticker_list:
      thread = Thread(target=get_format_info, args=(t, ticker_data))
      threads.append(thread)
      thread.start()

   # Once all threads complete, join them together 
   for th in threads:
      th.join()

   # Create and clean up the Dataframe
   ticker_df = pd.DataFrame(ticker_data, columns=["Ticker","Sector","Currency","MarketCap"]) 
   ticker_df = ticker_df[ticker_df["Currency"] != "NaN"]
   ticker_df = ticker_df.reset_index(drop=True)
   return ticker_df

In [ ]:
# Function to return a list of all tickers (first column elements)
def get_ticker_list (tickers_df):
     """
     get_ticker_list returns a Python list of tickers

     :param tickers_df: A Dataframe of tickers
     :return: A list of tickers
     """
     return tickers_df.iloc[:, 0].tolist()

# Gets weekly closes of all the stocks in a list of tickers
def get_weekly_closes (ticker_lst, start_date, end_date):
    """     
    get_weekly_closes performs data extraction to get weekly closing price info of stocks

    :param ticker_lst: List that holds all tickers
    :param start_date: start date for calculations
    :param end_date: end date for calculations
    :return: returns a Dataframe that holds the weekly closing price of each ticker on every week
    """
    #Define a dataframe to hold weekly close prices (checks every friday)
    cols = [] # list of Series to concat

    #Extract the weekly close prices and store them in the dataframe
    for i in ticker_lst: 
        ticker = yf.Ticker(i)
        data = ticker.history(start=start_date, end=end_date)
        data.index = pd.to_datetime(data.index) # ensure datetime index
        #last() takes the last trading price of the week
        series = data['Close'].resample('W-FRI').last()
        series.name = f"Close {i}" # name each column
        cols.append(series) # store weekly closes for each ticker
        
    # Concatenate all ticker series at once to avoid fragmentation
    weekly_closes = pd.concat(cols, axis=1)
    # Strip time
    weekly_closes.index = weekly_closes.index.strftime('%Y-%m-%d')
    return weekly_closes

# Creates a df with the (weekly) %change for each column
def get_percent_change (closes, start_date, end_date):
    """     
    get_percent_change performs calculations that determines the percent change of closing prices

    :param closes: Dataframe that holds closing prices
    :param start_date: start date for calculations
    :param end_date: end date for calculations
    :return: returns a Dataframe that holds percent changes of closing prices
    """
    cols = [] # list of Series to concat
    
    for i in closes:
        col_name = i[6:] # name each column by the ticker from "Close ---"
        # calculate %change
        series = closes[i].pct_change(fill_method=None) # Fill_method=None to hande delisted stocks
        series.name = f"% Change {col_name}" # name each column
        cols.append(series) # store %change for each ticker

    # Concatenate all ticker series at once to avoid fragmentation
    percent_change = pd.concat(cols, axis=1)
    return percent_change

# Calculate covariance, correlation, variance, standard deviation
def get_calculations(ticker_list, start_date, end_date):
    """     
    get_calculations performs calculations for covariance, correlation, variance, standard deviation

    :param ticker_list: list of all tickers
    :param start_date: start date for calculations
    :param end_date: end date for calculations
    :return: returns a dictionary that holds covariance, correlation, standard deviation, and variance 
    """
    weekly_closes = get_weekly_closes(ticker_list, start_date, end_date)
    weekly_percent_change = get_percent_change(weekly_closes, start_date, end_date)
    covariance_matrix = {
        'Covariance': weekly_percent_change.cov(),
        'Correlation': weekly_percent_change.corr(),
        'Variance': weekly_percent_change.var(),
        'Std_Dev': weekly_percent_change.std()}
    return covariance_matrix

info_df = format_tickers("Tickers.csv") #NOTE: CHANGE FILE NAME BEFORE SUBMITTING
ticker_list = (get_ticker_list(info_df)) # List of all tickers
primary_calculations = get_calculations(ticker_list, returns_start, returns_end) # Covariance matrix
"""
# Access each piece like:
display(primary_calculations['Covariance'])
display(primary_calculations['Std_Dev'])
"""

In [ ]:
filtered_metrics = primary_calculations
# Remove the % Change prefix from each index label
def strip_prefix(index_obj):
    cleaned = []
    for name in index_obj:
        s = str(name)

        # % Change is 9 characters long so check first 9 characters only
        if len(s) > 9 and s[0:9] == "% Change ":
            s = s[9:]   #So here we drop the first 9 characters

        cleaned.append(s)
    return cleaned

filtered_cov = filtered_metrics["Covariance"].mean(axis=1)
filtered_cov_tickers = strip_prefix(filtered_cov.index)

# Graphs it out
plt.figure(figsize=(30,5))
plt.scatter(filtered_cov_tickers, filtered_cov.values, s=100)
plt.xticks(rotation=90, fontsize=6)
plt.ylabel("Average Covariance")
plt.title("Initial Filtered Portfolio: Average Covariance per Ticker")
plt.tight_layout()
plt.show()

In [ ]:
# This is the function we want to minimize, aka the minimum variance function
def port_variance(weights, cov_matrix):
    """     
    port_variance is the function that calculates the variance of a portfolio. It performs 
    dot product/matrix multiplication on the weights and covariance matrixes. 

    :param weights: an array that represents the weight of each asset
    :param cov_matrix: a 2D matrix that represents the covariance between each asset 
    :return: variance of the portfolio
    """
    weights_col = weights.reshape(-1, 1) # Turns into column vector
    port_var = np.dot(weights_col.transpose(), (np.dot(cov_matrix, weights_col))) # Doing dot product 
    return port_var[0][0]

In [ ]:
# Primary minimization, there is bounds in this 
def primary_minimization(cov_matrix):
    """     
    primary_minimization is the function that finds the weightings that result in the mimimum variance. 
    It performs this using scipy optimization. This perimary version does not consider bounds. 

    :param cov_matrix: a 2D matrix that represents the covariance between each asset 
    :return: returns the minimum variance and the weightings associated with that
    """
    num_assets = cov_matrix.shape[0]
    initial_weight = [1/num_assets] * num_assets # The initial guess of the weights

    constraint = {
        'type':'eq', # Constraint type is equality
        'fun': lambda w: sum(w) - 1 # The function's weight's must sum to 1
        }
    
    weight_bounds = [(0, 1)] * num_assets # Does not allow short selling
    
    # Finds the resilt of the minimization of the port_variance function, using the initial guess, keeping the cov_matrix constant using the SLSQP method, and with the above listed constraint
    result = minimize(fun=port_variance, x0=initial_weight, args=(cov_matrix,), method='SLSQP', bounds=weight_bounds, constraints=constraint)
    return result.fun, result.x

pd_cov_matrix = primary_calculations['Covariance']
numpy_cov_matrix = pd_cov_matrix.to_numpy() # Matrix of covariances of assets
primary_var, primary_weights = primary_minimization(numpy_cov_matrix)

In [ ]:
# The secondary optimization that includes the bounds 
def secondary_minimization(cov_matrix):
    """     
    secondary_minimization is the function that finds the weightings that result in the mimimum variance while considering the bounds.
    That is, it ensures the weightings 

    :param cov_matrix: a 2D matrix that represents the covariance between each asset 
    :return: returns the minimum variance and the weightings associated with that
    """
    num_assets = len(cov_matrix[0]) 
    initial_weight = [1/num_assets] * num_assets # The initial guess of the weights

    min_weight = (100/(2*num_assets))/100 # Do not need to include portfolio value, because 1 is the portfolio value (and sum of weights)
    max_weight = 0.15 # Same as above

    constraint = {
        'type':'eq', # Constraint type is equality
        'fun': lambda w: sum(w) - 1 # The function's weight's must sum to 1
        }
    
    weight_bounds = [(min_weight, max_weight)] * num_assets
    
    # Finds the resilt of the minimization of the port_variance function, using the initial guess, keeping the cov_matrix constant using the SLSQP method, and with the above listed constraint
    result = minimize(fun=port_variance, x0=initial_weight, args=(cov_matrix,), method='SLSQP', bounds = weight_bounds, constraints=constraint)
    return result.fun, result.x

In [ ]:
# Assign weights to the portfolio consisting of ALL stocks and sort from most important to least
info_df['weight'] = primary_weights
ordered_info_df = info_df.copy().sort_values('weight', ascending=False).reset_index(drop=True)

# Start with top 25 by weight
top_25 = ordered_info_df.iloc[:25].copy()

# Check if there is at least one small cap (< 2B)
small_caps_in_top25 = top_25[top_25['MarketCap'] < 2_000_000_000]

# Check if there is at least one large cap (> 10B)
large_caps_in_top25 = top_25[top_25['MarketCap'] > 10_000_000_000]

# If we have both, nothing to change
if small_caps_in_top25.empty or large_caps_in_top25.empty:
    # Start with top 24 to replace missing cap types if needed
    top_24 = ordered_info_df.iloc[:24].copy()
    
    # Add a small-cap if missing
    if small_caps_in_top25.empty:
        lowest_index_small_cap = ordered_info_df[ordered_info_df['MarketCap'] < 2_000_000_000].iloc[0:1]
        top_24 = pd.concat([top_24, lowest_index_small_cap], ignore_index=True)
    
    # Add a large-cap if missing
    if large_caps_in_top25.empty:
        lowest_index_large_cap = ordered_info_df[ordered_info_df['MarketCap'] > 10_000_000_000].iloc[0:1]
        top_24 = pd.concat([top_24, lowest_index_large_cap], ignore_index=True)
    
    # Now top_24 (or top_24 + 1 or 2) becomes the final top 25
    ordered_top_25 = top_24.iloc[:25].copy()  # ensure exactly 25 rows
else:
    ordered_top_25 = top_25.copy()

# Update ordered_info_df with final top 25
ordered_info_df = ordered_top_25

# Display the final top 25
print("===========================================================================================")
print("These are the top 25 stocks that will be sampled to determine the most risk-free portfolio:")
print("===========================================================================================")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ordered_info_df)


In [ ]:
# Code that creates a portfolio that matches the requirements

# Determines all the indexes in a portfolio that are large cap stocks
def is_lg_cap(portfolio):
    """
     is_lg_cap determines all the indexes in a portfolio that are large cap stocks

     :param portfolio: A list of Series objects that contain a stock's info 
     :return: A list of indexes of ticker's that are large cap
     """
    lg_cap = []
    for i in range(len(portfolio)):
        if portfolio[i]['MarketCap'] > 10_000_000_000:
            lg_cap.append(i)
    return lg_cap

# Determines all the indexes in a portfolio that are small cap stocks
def is_sm_cap(portfolio):
    """
     is_sm_cap determines all the indexes in a portfolio that are small cap stocks

     :param portfolio: A list of Series objects that contain a stock's info 
     :return: A list of indexes of ticker's that are small cap
     """
    sm_cap = []
    for i in range(len(portfolio)):
        if portfolio[i]['MarketCap'] < 2_000_000_000:
            sm_cap.append(i)
    return sm_cap

# Determines if an individual stock is a large market cap
def is_lg(row):
    return row['MarketCap'] > 10_000_000_000

# Determines if an individual stock is a small market cap
def is_sm(row):
    return row['MarketCap'] < 2_000_000_000

# Determines the index of the least important stock. Least important in this case is the one with the lowest weighting in the ordered list, but
# if the least important is either the ONLY SMALL or LARGE cap stock then the second least important stock is now designated the least important
def find_least_imp(portfolio, lg_indxs, sm_indxs):
    """ 
    find_least_imp Determines the index of the least important stock. Least important in this case is the one with the lowest weighting in the ordered list, but 
    if the least important is either the ONLY SMALL or LARGE cap stock then the second least important stock is now designated the least important

    :param portfolio: A list of stock data in Series format
    :param lg_indxs: A list of all indexes that hold large cap stocks
    :param sm_indxs: A list of all indexes that hold small cap stocks
    :return: integer that represents the index that is desginated least important
    """

    only_large_idx = None
    only_small_idx = None

    # Determines the protected indexes
    if len(lg_indxs) == 1:
        only_large_idx = lg_indxs[0]
    if len(sm_indxs) == 1:
        only_small_idx = sm_indxs[0]

    for i in range(len(portfolio) - 1, -1, -1):
        if i != only_large_idx and i != only_small_idx:
            return i

# Creates a portfolio that is valid, aside from the 40% sector restriction
def create_portfolio(size):
    """ 
    create_portfolio creates a portfolio containing both small and large cap

    :param size: the size of the portfolio
    :return: a list of tickers that are in the portfolio, or none if it cannot create a portfolio that satisfies all requirements
    """

    portfolio = [] # Will be a list of Series
    i = 0
    ticker_only = []
    
    # Creates preliminary portfolio 
    while len(portfolio) < size:
        if i >= len(ordered_info_df):
            return None
        cur = ordered_info_df.iloc[i]
        portfolio.append(cur)
        i += 1

    lg_idxs = is_lg_cap(portfolio)
    sm_idxs = is_sm_cap(portfolio)
    
    for stock in portfolio:
        ticker_name = stock["Ticker"]
        ticker_only.append(ticker_name)
    
    return ticker_only

In [ ]:
def compute_sector_weights(tickers, weights, ordered_info_df):
    """
    Calculate how much of the portfolio, by weight, belongs to each sector.
    :param tickers: list of ticker names 
    :param weights: list of weights from the optimizer 
    :param info_df: dataframe that includes Ticker and Sector columns
    :return: the sector weight total
    """

    # Make lookup by ticker
    info_by_ticker = ordered_info_df.set_index("Ticker")

    sector_totals = {}  # Holds sum of weights for each sector

    # Loop through
    for ticker, wt in zip(tickers, weights):
        sector = info_by_ticker.loc[ticker, "Sector"]
        
        # Add weight to appropriate sector
        if sector not in sector_totals:
            sector_totals[sector] = 0.0
        sector_totals[sector] += float(wt)

    return sector_totals

def optimize_with_sector_limit(initial_tickers, returns_start, returns_end, ordered_info_df):
    """
    optimize_with_sector_limit will run optimization and enforce 40% sector-by-value constraint by replacing the
    smallest-weight stock in the overweight sector until constraints are satisfied or no replacements are possible
    :param initial_tickers: The list of ticker symbols that is the starting portfolio before optimization
    :param returns_start: A date
    :param returns_end: A date
    :param info_df: A DataFrame containing the Ticker and Sector columns, in our case its gonna be ordered_info_df
    """

    current_tickers = list(initial_tickers)
    used_tickers = set(current_tickers)

    # Maximum range is 20, only go through 20 iterations before stopping to prevent runtime loss
    for n in range(20):
        # Get values needed for secondary optimization
        calc_df = get_calculations(current_tickers, returns_start, returns_end)
        cov_np = calc_df["Covariance"].to_numpy()

        portfolio_variance, weights = secondary_minimization(cov_np)
        weights = np.array(weights).flatten() 

        # Get each sectors weights
        sector_weights = compute_sector_weights(current_tickers, weights, ordered_info_df)

        # Find the sector with the largest weight, if this one is valid that all others are valid
        heaviest_sector = None
        heaviest_weight = -1.0
        for sector, w in sector_weights.items():
            if w > heaviest_weight:
                heaviest_sector = sector
                heaviest_weight = w

        # Check if the heaviest weight is under the max, if so return the information
        if heaviest_weight <= 0.4:
            return current_tickers, portfolio_variance, weights

        # The following finds the least (smallest) weighted, aka least important, stock of the heaviest sector
        info_indexed = ordered_info_df.set_index("Ticker")

        smallest_weight = 100000 # Placeholder
        smallest_index = None

        for i in range(len(current_tickers)):
            sector_name = info_indexed.at[current_tickers[i], "Sector"]
            if sector_name == heaviest_sector:
                if weights[i] < smallest_weight:
                    smallest_weight = weights[i]
                    smallest_index = i

        # Get a replacement ticker to replace the least important heaviest sector 
        replacement_ticker = None
        for row in ordered_info_df.itertuples(index=False):
            candidate = row.Ticker
            if candidate not in used_tickers:
                replacement_ticker = candidate
                break

        # If no new ticker available then stop trying to fix portfolio
        if replacement_ticker is None:
            return None, None, None

        # Replace smallest-weight stock in overweight sector
        used_tickers.add(replacement_ticker)
        current_tickers[smallest_index] = replacement_ticker

    # End loop fallback once we go over iterations
    return None, None, None

In [ ]:
all_ports = []
all_variance = []
all_weights = []
count = 0

# Go through all the 15-25 sized portfolios and then find the one that results in minimum variance. 
while (count + 10) < 26:

    size = count + 10
    base_port = create_portfolio(size)

    if base_port is None:
        print(f"Failed to build portfolio of size {count+10}, skipping.") # portfolio requirements failed
        count += 1
        continue

    final_port, final_var, final_w = optimize_with_sector_limit(base_port, returns_start, returns_end, ordered_info_df)
    if final_port is None:
        print(f"Sector constraint failed for portfolio size {count+10}, skipping.") # portfolio requirements failed
        count += 1
        continue

    all_ports.append(final_port)
    all_variance.append(final_var)
    all_weights.append(final_w)
    count += 1

if not all_variance:
    print("No valid portfolios were generated for some reason. Please check ticker csv.") # no portfolios could be generated based on the constraints
else:
    smallest_var = min(all_variance)
    index = all_variance.index(smallest_var)
    target = [smallest_var, all_ports[index], all_weights[index]]
    print("================================================================================================================================================")
    print(f"The smallest variance found is {target[0]} which is determined from the following portfolio:'\n{target[1]},\nat the following weights:\n{target[2]}.")
    print("================================================================================================================================================")


In [ ]:
def get_close_prices_and_rate(tickers, target_date, end_date):
    """
    get_close_prices_and_rate finds the closing price of tickers on a date, and the exchange rate on a date

    :param tickers: list of tickers
    :param target_data: the day of price we want, normally most recent business day
    :param end_date: the day after, as yfinance is not inclusive
    :return: a Series that contains the target days close price
    :return: the USD to CAD exchange rate
    """
    data = yf.download(tickers, start=target_date, end=end_date)["Close"] # Download data for tickers
    close_prices = data.iloc[0]

    exchange_rate = yf.download("CAD=X", start=target_date, end=end_date)["Close"] # Download data for exchange rate
    exchange_rate = exchange_rate.iloc[0]

    return close_prices, exchange_rate.item()

def purchase_flat_fee(df, budget, exchange_rate):
    """
    purchase_flat_fee transforms a Dataframe to include the amount of shares bought with a flat fee, and the value of that

    :param df: The Dataframe that will be transformed
    :param budget: An Integer representing the budget
    :param exchange_rate: A series of the USD-CAD exchange rate
    :return: Transformed Dataframe
    """

    df["Shares Bought Flat Fee"] = (df["Weight"] * (budget - (2.5*exchange_rate))) / df["Price"]
    df["Flat Fee Worth"] = df["Shares Bought Flat Fee"] * df["Price"]

    return df 

def purchase_variable_fee(df, budget, exchange_rate):
    """
    purchase_variable_fee transforms a Dataframe to include the amount of shares bought with the share-dependent rate

    :param df: The Dataframe that will be transformed
    :param budget: An Integer representing the budget
    :param exchange_rate: A series of the USD-CAD exchange rate
    :return: Transformed Dataframe
    """
    df["Shares w/o Fee"] = (df["Weight"] * budget) / df["Price"]
    total_shares = df["Shares w/o Fee"].sum()
    
    # Calulate the variable fee
    variable_fee_usd = total_shares * 0.001
    variable_fee_cad = variable_fee_usd * exchange_rate

    adjusted_budget = budget - variable_fee_cad # adjust the budget to how much we would be able to after the trading fee
    df["Shares Bought Variable Fee"] = (df["Weight"] * adjusted_budget) / df["Price"] # buy stocks using the adjusted budget
    df["Variable Fee Worth"] = df["Shares Bought Variable Fee"] * df["Price"]

    return df

def ideal_shares(df):
    """ 
    ideal_shares determines which fee strategy, flat or variable, is the most optimal and transforms Dataframe accordingly

    :param df: The Dataframe that will be transformed
    :return: Transformed Dataframe
    """
    sum_flat_fee = df["Flat Fee Worth"].sum()
    sum_variable_fee = df["Variable Fee Worth"].sum()

    if (sum_flat_fee < sum_variable_fee):
        df.drop(["Shares Bought Flat Fee", "Flat Fee Worth", "Shares w/o Fee"], axis=1, inplace=True)
        df.rename(columns={"Shares Bought Variable Fee":"Shares", "Variable Fee Worth":"Value"}, inplace=True)
    else:
        df.drop(["Shares Bought Variable Fee", "Variable Fee Worth", "Shares w/o Fee"], axis=1, inplace=True)
        df.rename(columns={"Shares Bought Flat Fee":"Shares", "Flat Fee Worth":"Value"}, inplace=True)
    
    return df 

def add_currency(df_small, df_large):
    """ 
    add_currency adds the currency type onto the Dataframe

    :param df_small: The Dataframe that holds the shares bought
    :param_df_large: The Dataframe the holds the Currency's
    :return: Transformed Dataframe
    """

    df_with_currency = df_small.merge(df_large[["Ticker", "Currency"]], on="Ticker", how="left")
    return df_with_currency

def convert_closing(df, exchange_rate):
    """
    convert_closing converts the price of any stock thats in USD to CAD

    :param df: Dataframe to be transformed
    :param exchange_rate: The USD-CAD rate
    :return: Dataframe with converted prices
    """

    df.loc[df["Currency"] == "USD", "Price"] = df["Price"] * exchange_rate
    return df

In [ ]:
temp_df = pd.DataFrame({
    "Ticker": target[1],
    "Weight": target[2]
})

target_date = purchase_date
end_date = purchase_date + timedelta(days=1) # yfinance is exclusive 
closing, usd_cad_rate = get_close_prices_and_rate(target[1], "2025-11-26", "2025-11-27")
temp_df = temp_df.merge(closing.rename("Price"), left_on="Ticker", right_index=True, how="left")
temp_df = add_currency(temp_df,ordered_info_df)
temp_df_cad = convert_closing(temp_df, usd_cad_rate)
temp_df_cad = purchase_flat_fee(temp_df_cad, 1_000_000, usd_cad_rate)
temp_df_cad = purchase_variable_fee(temp_df_cad, 1_000_000, usd_cad_rate)
Portfolio_Final = ideal_shares(temp_df_cad)
Portfolio_Final = Portfolio_Final[["Ticker", "Price", "Currency", "Shares", "Value", "Weight"]]
print("===========================================================================")
print("This is the final portfolio with basic info for each stock: ")
print("===========================================================================")
print("")
Portfolio_Final

In [ ]:
sum_weight = Portfolio_Final["Weight"].sum() # Doesn't return 1 properly because floats can't sum to exact integers
sum_shares = Portfolio_Final["Shares"].sum()
sum_value = Portfolio_Final["Value"].sum()
print("=======================================================================================================================================================")
print(f"The sum of the weights is {sum_weight}, the total value of the portfolio minus the fees is ${sum_value}, and the sum of the shares is {sum_shares}.")
print("=======================================================================================================================================================")

In [ ]:
filtered_metrics = primary_calculations
final_tickers = Portfolio_Final["Ticker"].tolist()
cov_matrix = filtered_metrics["Covariance"].copy()
cov_matrix.index = cov_matrix.index.str.replace("% Change ", "", regex=False)
final_cov = cov_matrix.loc[final_tickers].mean(axis=1)

# Graphs it out
plt.figure(figsize=(30,5))
plt.scatter(final_tickers, final_cov.values, s=100, color="orange")
plt.xticks(rotation=90, fontsize=14)
plt.ylabel("Average Covariance", fontsize=18)
plt.title("Final Portfolio: Average Covariance per Ticker", fontsize=22)
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
clean_var_index = strip_prefix(filtered_metrics["Variance"].index)
variance_series = filtered_metrics["Variance"].copy()
variance_series.index = clean_var_index
final_var = variance_series.loc[final_tickers]

# Graphs it out with overlay
plt.figure(figsize=(30,5))
plt.scatter(variance_series.index, variance_series.values,s=100, alpha=0.35, color="gray", label="All N Tickers")
plt.scatter(final_tickers, final_var.values,s=120, alpha=0.95, color="orange", edgecolors="black", label="Final Portfolio")
plt.xticks(rotation=90, fontsize=14)
plt.ylabel("Variance", fontsize=18)
plt.title("Variance Comparison", fontsize=24)
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(fontsize=18)
plt.tight_layout(pad=3.0)
plt.show()